In [11]:
import numpy as np
import random
import networkx as nx
import csv
import scipy.optimize
import functools

G_train = nx.Graph()
G_test = nx.Graph()
count = 0
ran = []
for i in range(1099):
    ran.append(random.randint(0,5))

k = 0
file = open('amazon-meta_filtered_ground_truth.txt')
for line in file:
    # line = line:[:-1]
    line = line[:-2]
    cell = line.split(',')
    count += len(cell)-1
    G_train.add_node(cell[0])
    if len(cell) > 1:
        for i in range(len(cell)-1):
            if i != ran[k]:
                G_train.add_edge(cell[0], cell[i+1])
    k += 1
file.close()

k = 0
file = open('amazon-meta_filtered_ground_truth.txt')
for line in file:
    # line = line:[:-1]
    line = line[:-2]
    cell = line.split(',')
    G_test.add_node(cell[0])
    if ran[k]+1 <= len(cell)-1:
        if G_train.has_edge(cell[0], cell[ran[k]+1]) == False:
            G_test.add_edge(cell[0], cell[ran[k]+1])
    k += 1
file.close()

print G_train.number_of_nodes()
print G_train.number_of_edges()
print G_test.number_of_nodes()
print G_test.number_of_edges()
print count

1099
608
1099
109
850


In [12]:
print G_train.nodes()

['0553374966', '1881539148', '0876282400', '0252069285', '1563085313', '0966269616', '0916685896', '0738703443', '0671761927', '1588320022', '0803971451', '1555703429', '0837194857', '0963158945', '0898797470', '0762706600', '0810836939', '1582970106', '1861001401', '1862100292', '0028638352', '0313250235', '0761978674', '1882926668', '1880124092', '0810828553', '0802138624', '0835219259', '0898797780', '0425170543', '0521789486', '0399525777', '1563081865', '1560009861', '0870202154', '0513022104', '089793217X', '1563086662', '0815607830', '0935012230', '0838907970', '0312241429', '0786411546', '0786408677', '0898798272', '089879398X', '0965768643', '0838907873', '1577311000', '0812042689', '1563089904', '1563088541', '1574442767', '1555704026', '1567500935', '188018401X', '0811818802', '1579500005', '0787956791', '0838908365', '0873493400', '0486229181', '0471353124', '087975916X', '1564965120', '0275946606', '1563086247', '0070230005', '0898797543', '0810845105', '1563086182', '0879

In [13]:
ID_to_Index = {}
Index_to_ID = {}
for i in range(G_train.number_of_nodes()):
    ID_to_Index[G_train.nodes()[i]] = i
    Index_to_ID[i] = G_train.nodes()[i]
e = G_train.edges()
train_edges = []
for i in range(G_train.number_of_edges()):
    train_edges.append((ID_to_Index[e[i][0]], ID_to_Index[e[i][1]]))

In [14]:
with open("amazon-meta_item_info.csv", "r") as f:
    reader = csv.reader(f,delimiter='^')
    node_info  = list(reader)

node_info = node_info[1:]
IDs = [element[0] for element in node_info]
title = [element[1] for element in node_info]

In [15]:
# number of reviews
reviews_num_similarity = []

counter = 0
for i in xrange(len(train_edges)):
#for i in range(1):
    source = Index_to_ID[train_edges[i][0]]
    target = Index_to_ID[train_edges[i][1]]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    # number of reviews similarity
    review_source = source_info[4]
    review_target = target_info[4]
    
    reviews_num_similarity.append(np.absolute(int(review_source)-int(review_target)))
    
    counter += 1
    if counter % 10000 == True:
        print counter, "training examples processsed"
print "Done"

1 training examples processsed
Done


In [16]:
# same number of words in detailed category
category_similarity = []

counter = 0
for i in xrange(len(train_edges)):
#for i in range(1):
    source = Index_to_ID[train_edges[i][0]]
    target = Index_to_ID[train_edges[i][1]]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    # same number of words in detailed category
    category_source = source_info[6+int(review_source)*4:]
    category_target = target_info[6+int(review_target)*4:]
    
    category_similarity.append(len(set(category_source).intersection(set(category_target))))
    
    counter += 1
    if counter % 10000 == True:
        print counter, "training examples processsed"
print "Done"

1 training examples processsed
Done


In [17]:
print len(train_edges)
print len(category_similarity)

608
608


In [18]:
f = open('amazon-meta_item_item_1.txt', 'w')
e = G_train.edges()
for i in range(G_train.number_of_edges()):
    f.write("%s,%s,%s,%s\n" % (train_edges[i][0],train_edges[i][1],reviews_num_similarity[i], category_similarity[i]))
f.close()

In [19]:
e = G_test.edges()
test_edges = []
for i in range(G_test.number_of_edges()):
    test_edges.append((ID_to_Index[e[i][0]], ID_to_Index[e[i][1]]))

In [20]:
# same number of words in detailed category
category_similarity = []

counter = 0
for i in xrange(len(test_edges)):
#for i in range(1):
    source = Index_to_ID[test_edges[i][0]]
    target = Index_to_ID[test_edges[i][1]]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    # same number of words in detailed category
    category_source = source_info[6+int(review_source)*4:]
    category_target = target_info[6+int(review_target)*4:]
    
    category_similarity.append(len(set(category_source).intersection(set(category_target))))
    
    counter += 1
    if counter % 10000 == True:
        print counter, "testing examples processsed"
print "Done"

1 testing examples processsed
Done


In [21]:
# number of reviews
reviews_num_similarity = []

counter = 0
for i in xrange(len(test_edges)):
#for i in range(1):
    source = Index_to_ID[test_edges[i][0]]
    target = Index_to_ID[test_edges[i][1]]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    # number of reviews similarity
    review_source = source_info[4]
    review_target = target_info[4]
    
    reviews_num_similarity.append(np.absolute(int(review_source)-int(review_target)))
    
    counter += 1
    if counter % 10000 == True:
        print counter, "testing examples processsed"
print "Done"

1 testing examples processsed
Done


In [22]:
f = open('amazon-meta_item_item_2.txt', 'w')
e = G_test.edges()
for i in range(G_test.number_of_edges()):
    f.write("%s,%s,%s,%s\n" % (ID_to_Index[e[i][0]],ID_to_Index[e[i][1]],reviews_num_similarity[i], category_similarity[i]))
f.close()